In [1]:
# G2M insight for Cab Investment firm
import pandas as pd
import numpy as np
import datetime

ride_data = pd.read_csv('~/Downloads/Cab_Data.csv')
city_data = pd.read_csv('~/Downloads/City.csv')
customer_data = pd.read_csv('~/Downloads/Customer_ID.csv')
transaction_data = pd.read_csv('~/Downloads/Transaction_ID.csv')


In [2]:
#------ Data exploration ------
# -- Cab data --
'''
Data for transaction of 2 cab companies.
Every ride with a Yellow/Pink cab in a City, total KM travelled, the price charged by the cab driver and the actual
cost of the trip.
'''

print(ride_data.head())
print(ride_data.info())

## Total number of data points = 359392; Total no. of features = 7
## No missing data
## 'Date of Travel' - is int64 datatype. Excel converted date into a 5-digit serial number.
##                    The 5-digit number represents the number of days since 1899/12/30. [Check bookmarked webpage.]

# Converting 'Date of Travel' to datetime format
excel_dates = ride_data['Date of Travel']
new_date_col = []
for d in excel_dates:
    temp = datetime.date(1899,12,30) + datetime.timedelta(float(d))
    new_date_col.append(temp.strftime('%Y-%m-%d'))
    
ride_data['Travel_Date'] = new_date_col

# Shift Travel Data column to first
first_col = ride_data.pop('Travel_Date')
ride_data.insert(0, 'Travel_Date', first_col)
# print(ride_data.head())

   Transaction ID  Date of Travel   Company        City  KM Travelled  \
0        10000011           42377  Pink Cab  ATLANTA GA         30.45   
1        10000012           42375  Pink Cab  ATLANTA GA         28.62   
2        10000013           42371  Pink Cab  ATLANTA GA          9.04   
3        10000014           42376  Pink Cab  ATLANTA GA         33.17   
4        10000015           42372  Pink Cab  ATLANTA GA          8.73   

   Price Charged  Cost of Trip  
0         370.95       313.635  
1         358.52       334.854  
2         125.20        97.632  
3         377.40       351.602  
4         114.62        97.776  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359392 entries, 0 to 359391
Data columns (total 7 columns):
Transaction ID    359392 non-null int64
Date of Travel    359392 non-null int64
Company           359392 non-null object
City              359392 non-null object
KM Travelled      359392 non-null float64
Price Charged     359392 non-null float64
Cost of

In [3]:
# -- City data --
'''
Data of population and total number of cab users in each city.
Users - Assuming users of all types of cabs (including the Yellow and Pink cabs)
'''
print(city_data.head())
print(city_data.info())

## Total no. of entries = 20; No. of features = 3
## No missing values


             City   Population      Users
0     NEW YORK NY   8,405,837    302,149 
1      CHICAGO IL   1,955,130    164,468 
2  LOS ANGELES CA   1,595,037    144,132 
3        MIAMI FL   1,339,155     17,675 
4  SILICON VALLEY   1,177,609     27,247 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
City          20 non-null object
Population    20 non-null object
Users         20 non-null object
dtypes: object(3)
memory usage: 560.0+ bytes
None


In [4]:
# -- Customer data --
'''
Data of customers subscribed with the two cab companies.
Their Gender, Customer ID, Age and Income (USD/Month) given here.
'''
print(customer_data.info())
print(customer_data.head())
## Total no. of entries = 49171; No. of features = 4
## No missing values

# THIS DATA CAN BE JOINED WITH TRANSACTION DATA USING 'Customer ID'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49171 entries, 0 to 49170
Data columns (total 4 columns):
Customer ID           49171 non-null int64
Gender                49171 non-null object
Age                   49171 non-null int64
Income (USD/Month)    49171 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.5+ MB
None
   Customer ID Gender  Age  Income (USD/Month)
0        29290   Male   28               10813
1        27703   Male   27                9237
2        28712   Male   53               11242
3        28020   Male   23               23327
4        27182   Male   33                8536


In [21]:
# -- Transaction data -- 
'''
Data containing the mode of transaction for each ride in the Yellow/Pink cab.
'''
print(transaction_data.info())
print(transaction_data.head())
## Total no. of entries = 440098;  No. of features = 3
## No missing values

# CAN JOIN TRANSACTION AND CUSTOMER DATA USING 'Customer ID'


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440098 entries, 0 to 440097
Data columns (total 3 columns):
Transaction ID    440098 non-null int64
Customer ID       440098 non-null int64
Payment_Mode      440098 non-null object
dtypes: int64(2), object(1)
memory usage: 10.1+ MB
None
   Transaction ID  Customer ID Payment_Mode
0        10000011        29290         Card
1        10000012        27703         Card
2        10000013        28712         Cash
3        10000014        28020         Cash
4        10000015        27182         Card


In [11]:
# INNER JOIN BETWEEN THE RIDE DATA AND CITY DATA

ride_city_out = pd.merge(ride_data, city_data, how = 'inner', on = 'City')
print(ride_city_out.shape)
print(ride_city_out.head())
print(ride_city_out.isnull().any())

# SAN FRANCISCO CA - NO RIDE DATA AVAILABLE WITH THE YELLOW AND PINK CABS !!!!!!!!!!!!!!!!!!!!!!!
# ON DROPPING SAN FRANCISCO CA, NO NULL VALUES IN ALL COLUMNS ----------- DELETE THIS LINE

(359392, 10)
  Travel_Date  Transaction ID  Date of Travel   Company        City  \
0  2016-01-08        10000011           42377  Pink Cab  ATLANTA GA   
1  2016-01-06        10000012           42375  Pink Cab  ATLANTA GA   
2  2016-01-02        10000013           42371  Pink Cab  ATLANTA GA   
3  2016-01-07        10000014           42376  Pink Cab  ATLANTA GA   
4  2016-01-03        10000015           42372  Pink Cab  ATLANTA GA   

   KM Travelled  Price Charged  Cost of Trip Population     Users  
0         30.45         370.95       313.635   814,885    24,701   
1         28.62         358.52       334.854   814,885    24,701   
2          9.04         125.20        97.632   814,885    24,701   
3         33.17         377.40       351.602   814,885    24,701   
4          8.73         114.62        97.776   814,885    24,701   
Travel_Date       False
Transaction ID    False
Date of Travel    False
Company           False
City              False
KM Travelled      False
Price Ch

In [26]:
# INNER JOIN BETWEEN TRANSACTION AND CUSTOMER DATA
transaction_out_cust = pd.merge(transaction_data, customer_data, how = 'inner', on = 'Customer ID')

print(transaction_out_cust.head())
print(transaction_out_cust.shape)
print(transaction_out_cust.isnull().any())
# NO NULL VALUES AFTER JOINING TRANSACTION AND CUSTOMER DATA

   Transaction ID  Customer ID Payment_Mode Gender  Age  Income (USD/Month)
0        10000011        29290         Card   Male   28               10813
1        10351127        29290         Cash   Male   28               10813
2        10412921        29290         Card   Male   28               10813
3        10000012        27703         Card   Male   27                9237
4        10320494        27703         Card   Male   27                9237
(440098, 6)
Transaction ID        False
Customer ID           False
Payment_Mode          False
Gender                False
Age                   False
Income (USD/Month)    False
dtype: bool


In [33]:
# INNER JOIN BETWEEN ride_city_out and transaction_out_cust dataframes
Final_table = pd.merge(ride_city_out, transaction_out_cust, how = 'inner', on = 'Transaction ID')
print(Final_table.shape)
Final_table = Final_table.sort_values('Travel_Date', ascending = True)

# FIRST RECORDED DATE OF CAB RIDE: 2016-01-02
# LAST RECORDED DATE OF CAB RIDE: 2018-12-31
print(Final_table.isnull().sum())

# WHEN USED 'OUTER' JOIN, GOT NULL VALUES IN THE COLUMNS - 'Travel_Date', 'Date of Travel', 'Company', 'City',
#                               'KM Travelled', 'Price Charged', 'Cost of Trip', 'Population', 'Users'
# NO. OF ROWS WITH NULL VALUES = 80706
# HOWEVER WITH 'INNER' JOIN, THE NUMBER OF OBSERVATIONS DROP BY 80706
# I MIGHT HAVE LOST CERTAIN AMOUNT OF DATA OF CUSTOMERS (AGE, INCOME/MONTH, GENDER)
# THE DATA LOST DIDN'T HAVE ANY DETAILS ABOUT THE RIDE, BUT ONLY CUSTOMER ID, AGE AND GENDER ARE PRESENT

# TRANSACTION ID, CUSTOMER_ID AND PAYMENT MODE NOT USEFUL IN FURTHER ANALYSIS

(359392, 15)
Travel_Date           0
Transaction ID        0
Date of Travel        0
Company               0
City                  0
KM Travelled          0
Price Charged         0
Cost of Trip          0
Population            0
Users                 0
Customer ID           0
Payment_Mode          0
Gender                0
Age                   0
Income (USD/Month)    0
dtype: int64


In [42]:
# DROP THE UNNECESSARY COLUMNS
# Final_2 = Final_table.drop(['Transaction ID', 'Customer ID', 'Payment_Mode'], axis = 1, inplace = False)
# print(Final_2.shape)

print(Final_table.shape)
print(Final_table.isnull().any())

# ---- DELETE THIS CELL ----

# To check duplicate rows
# duplicated_rows = Modified.duplicated(subset = None, keep = 'first')
# No_dupl_modified = Modified.drop_duplicates(subset = None, keep = False)
# print(No_dupl_modified.shape)
# print(No_dupl_modified[No_dupl_modified.isnull()])

(359392, 15)
Travel_Date           False
Transaction ID        False
Date of Travel        False
Company               False
City                  False
KM Travelled          False
Price Charged         False
Cost of Trip          False
Population            False
Users                 False
Customer ID           False
Payment_Mode          False
Gender                False
Age                   False
Income (USD/Month)    False
dtype: bool


In [50]:
# ACQUIRING THIRD PARTY DATASET - US HOLIDAY DATASET (2004 - 2021)
usholiday_data = pd.read_csv('~/Downloads/US Holiday Dates (2004-2021).csv')
print(usholiday_data.info())
## Total no. of entries = 342; Total no. of features = 6
## No missing values
# print(usholiday_data.head())

# Subsetting the US Holiday dataset based on the start and end date of the cab ride data
start_date = '2016-01-02'
end_date = '2018-12-31'

usholiday_final = usholiday_data[(usholiday_data['Date'] >= start_date) & (usholiday_data['Date'] <= end_date)]
usholiday_final = usholiday_final.sort_values('Date', ascending = True).reset_index(drop = True)
print(usholiday_final.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 6 columns):
Date       342 non-null object
Holiday    342 non-null object
WeekDay    342 non-null object
Month      342 non-null int64
Day        342 non-null int64
Year       342 non-null int64
dtypes: int64(3), object(3)
memory usage: 16.1+ KB
None
         Date                      Holiday WeekDay  Month  Day  Year
0  2016-01-18  Martin Luther King, Jr. Day  Monday      1   18  2016
1  2016-02-14              Valentine’s Day  Sunday      2   14  2016
2  2016-02-15        Washington's Birthday  Monday      2   15  2016
3  2016-03-27               Western Easter  Sunday      3   27  2016
4  2016-05-01               Eastern Easter  Sunday      5    1  2016


In [53]:
# JOINING THE FINAL DATA AND US HOLIDAY DATA
# THIS DATA WILL BE USED LATER IN THE ANALYSIS
# FOR THAT CASE, USE INNER JOIN FOR THE BELOW MERGE
final_merged_data = pd.merge(Final_table, usholiday_final, how = 'outer', left_on = 'Travel_Date', right_on = 'Date', sort = True)
print(final_merged_data.shape)
print(final_merged_data.head())

# Dropping 'Date column'
final_merged_data_new = final_merged_data.drop('Date', axis = 1)
print(final_merged_data_new.shape)
## TOTAL NO. OF FEATURES = 17; TOTAL NO. OF OBSERVATIONS = 359854

# NEXT STEP, FILL OUT THE DATA INTAKE REPORT.
# FROM THAT, UNDERSTAND WHAT OTHER FEATURES ARE NEEDED.
# THEN START EDA BY DOING UNIVARIATE AND BIVARIATE ANALYSIS.

(359854, 21)
  Travel_Date  Transaction ID  Date of Travel     Company         City  \
0  2016-01-02        10000832           42371  Yellow Cab  NEW YORK NY   
1  2016-01-02        10000670           42371  Yellow Cab    DALLAS TX   
2  2016-01-02        10001234           42371  Yellow Cab   SEATTLE WA   
3  2016-01-02        10000845           42371  Yellow Cab  NEW YORK NY   
4  2016-01-02        10001232           42371  Yellow Cab   SEATTLE WA   

   KM Travelled  Price Charged  Cost of Trip   Population      Users  ...   \
0         29.29         907.43      369.0540   8,405,837    302,149   ...    
1         20.90         619.66      258.3240     942,908     22,157   ...    
2         39.60        1119.67      541.7280     671,238     25,063   ...    
3         17.92         561.71      253.7472   8,405,837    302,149   ...    
4         16.24         540.27      210.4704     671,238     25,063   ...    

   Payment_Mode Gender Age  Income (USD/Month)  Date Holiday WeekDay Mont

In [30]:
# Outer join between ride data and transaction data
ride_transaction = pd.merge(ride_data, transaction_data, on = 'Transaction ID', how = 'outer')
print(ride_transaction.info())
print(ride_transaction.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440098 entries, 0 to 440097
Data columns (total 10 columns):
Travel_Date       359392 non-null object
Transaction ID    440098 non-null int64
Date of Travel    359392 non-null float64
Company           359392 non-null object
City              359392 non-null object
KM Travelled      359392 non-null float64
Price Charged     359392 non-null float64
Cost of Trip      359392 non-null float64
Customer ID       440098 non-null int64
Payment_Mode      440098 non-null object
dtypes: float64(4), int64(2), object(4)
memory usage: 36.9+ MB
None
Travel_Date       80706
Transaction ID        0
Date of Travel    80706
Company           80706
City              80706
KM Travelled      80706
Price Charged     80706
Cost of Trip      80706
Customer ID           0
Payment_Mode          0
dtype: int64


In [29]:
# Inner join between transaction and customer data.
result = pd.merge(transaction_data, customer_data, on = 'Customer ID', how = 'inner')
print(result.head())
print(result.info())

result1 = pd.merge(ride_data.drop('Date of Travel', axis = 1), result, on = 'Transaction ID', how = 'outer')
print(result1.head())
print(result1.info())
print(result1.isna().sum())

   Transaction ID  Customer ID Payment_Mode Gender  Age  Income (USD/Month)
0        10000011        29290         Card   Male   28               10813
1        10351127        29290         Cash   Male   28               10813
2        10412921        29290         Card   Male   28               10813
3        10000012        27703         Card   Male   27                9237
4        10320494        27703         Card   Male   27                9237
<class 'pandas.core.frame.DataFrame'>
Int64Index: 440098 entries, 0 to 440097
Data columns (total 6 columns):
Transaction ID        440098 non-null int64
Customer ID           440098 non-null int64
Payment_Mode          440098 non-null object
Gender                440098 non-null object
Age                   440098 non-null int64
Income (USD/Month)    440098 non-null int64
dtypes: int64(4), object(2)
memory usage: 23.5+ MB
None
  Travel_Date  Transaction ID   Company        City  KM Travelled  \
0  2016-01-08        10000011  Pink Cab  AT